In [ ]:
# Install required packages
!pip install datasets
!pip install fasttext
!pip install numpy
!pip install pandas
!pip install scipy
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [ ]:
# Import relevant modules
from datasets import load_dataset
import re
from datasets import DatasetDict
import os
import fasttext
import numpy as np
import pandas
import scipy
import math
import csv
import tensorflow as tf
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr

In [ ]:
# Load the dataset
df = load_dataset("wikipedia", "20220301.en", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
# Preprocess the text
def preprocess(text):
    text = re.sub(r'[^\w\s]+', ' ', text)
    text = re.sub(r'[ \n]+', ' ', text)
    return text.strip().lower()

In [ ]:

train_dataset = df['train']

# Create the file path to store the preprocessed text
preprocessed_text_file = 'pt10000.txt'

# Set the counter as per the text file
counter = 1

# Open the file for writing
with open(preprocessed_text_file, 'w', encoding='utf-8') as f:
    # Iterate over the train dataset
    for data_point in train_dataset:
        text = data_point['text']

        # Perform any preprocessing steps on the text
        preprocessed_text = preprocess(text)

        # Write the preprocessed text to the file
        f.write(preprocessed_text + '\n')
        print("Progress: ", data_point['title'], counter)
        if(counter == 10000):
            break
        counter += 1

Progress:  Anarchism 1
Progress:  Autism 2
Progress:  Albedo 3
Progress:  A 4
Progress:  Alabama 5
Progress:  Achilles 6
Progress:  Abraham Lincoln 7
Progress:  Aristotle 8
Progress:  An American in Paris 9
Progress:  Academy Award for Best Production Design 10
Progress:  Academy Awards 11
Progress:  Actrius 12
Progress:  Animalia (book) 13
Progress:  International Atomic Time 14
Progress:  Altruism 15
Progress:  Ayn Rand 16
Progress:  Alain Connes 17
Progress:  Allan Dwan 18
Progress:  Algeria 19
Progress:  List of Atlas Shrugged characters 20
Progress:  Anthropology 21
Progress:  Agricultural science 22
Progress:  Alchemy 23
Progress:  Alien 24
Progress:  Astronomer 25
Progress:  ASCII 26
Progress:  Austin (disambiguation) 27
Progress:  Animation 28
Progress:  Apollo 29
Progress:  Andre Agassi 30
Progress:  Austroasiatic languages 31
Progress:  Afroasiatic languages 32
Progress:  Andorra 33
Progress:  Arithmetic mean 34
Progress:  American Football Conference 35
Progress:  Animal Far

In [ ]:
# Train the model (SkipGram 100 dimensions)
model = fasttext.train_unsupervised("pt10000.txt")

In [ ]:
# Save the model
model.save_model("model10000.bin")

In [ ]:
# Load the model
model10000 = fasttext.load_model("model10000.bin")

In [ ]:
# Get the word vector for a word
word1 = 'king'
word2 = 'man'
word3  = 'woman'

vectorOne1 = model10000.get_word_vector(word1)
vectorTwo1 = model10000.get_word_vector(word2)
vectorThree1 = model10000.get_word_vector(word3)

# Calculate the cosine similarity between two words
similarityOne1 = np.dot(vectorOne1, vectorTwo1) / (np.linalg.norm(vectorOne1) * np.linalg.norm(vectorTwo1))
similarityTwo1 = np.dot(vectorOne1, vectorThree1) / (np.linalg.norm(vectorOne1) * np.linalg.norm(vectorThree1))

# Print the cosine similarity
print(f'The cosine similarity between "{word1}" and "{word2}" using model1 is {similarityOne1:.2f}')
print(f'The cosine similarity between "{word1}" and "{word3}" using model1 is {similarityTwo1:.2f}')

The cosine similarity between "king" and "man" using model1 is 0.31
The cosine similarity between "king" and "woman" using model1 is 0.30


In [ ]:
# Load the STS Benchmark dataset
sts_dataset = tf.keras.utils.get_file(
    fname="Stsbenchmark.tar.gz",
    origin="http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz",
    extract=True)

sts_dev = pandas.read_table(
    os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-dev.csv"),
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["sim", "sent_1", "sent_2"])

sts_test = pandas.read_table(
    os.path.join(
        os.path.dirname(sts_dataset), "stsbenchmark", "sts-test.csv"),
    quoting=csv.QUOTE_NONE,
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["sim", "sent_1", "sent_2"])

# Cleanup some NaN values in sts_dev
sts_dev = sts_dev[[isinstance(s, str) for s in sts_dev['sent_2']]]
sts_test = sts_test[[isinstance(s, str) for s in sts_test['sent_2']]]

In [ ]:
# Preprocess a sentence by converting it to lowercase and splitting into words
def preprocess_sentence(sentence):
  return sentence.lower().split()

# Calculate the sentence embedding using average word embeddings
def get_sentence_embedding(model, sentence):
  return model.get_sentence_vector(sentence)

# Evaluate the FastText model on the STS Benchmark data
def run_sts_benchmark(model, sts_data):
  scores = []
  for sent1, sent2, label in zip(sts_data["sent_1"], sts_data["sent_2"], sts_data["sim"]):
    # Preprocess sentences
    preprocessed_sent1 = preprocess_sentence(sent1)
    preprocessed_sent2 = preprocess_sentence(sent2)

    # Get sentence embeddings one at a time
    sentence1_embedding = get_sentence_embedding(model, " ".join(preprocessed_sent1))
    sentence2_embedding = get_sentence_embedding(model, " ".join(preprocessed_sent2))

    # Calculate cosine similarity
    cosine_similarity = 1 - cosine(sentence1_embedding, sentence2_embedding)  # Higher value means more similar

    scores.append(cosine_similarity)
  return scores



# Load STS Benchmark data
sts_data = sts_test

# Run STS Benchmark evaluation
scores = run_sts_benchmark(model100000, sts_data)

# Calculate Pearson correlation coefficient
pearson_correlation, pValue = pearsonr(scores, sts_data["sim"])

# Print the results
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(pearson_correlation, pValue))

Pearson correlation coefficient = 0.4458044605673449
p-value = 2.6879109796957862e-68
